<a href="https://colab.research.google.com/github/anilbhatt1/Deep_Learning_EVA4_Phase2/blob/master/EVA4P2_S8_SRGAN_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

Sun Oct 11 07:24:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install torch==1.5.1+cu92 torchvision==0.6.1+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import zipfile
from zipfile import ZipFile
from pathlib import Path
from time import time
from datetime import datetime 
from __future__ import print_function
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from PIL import Image
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import cv2
import pkgutil
import importlib
import os
import random
%matplotlib inline
%config IPCompleter.greedy=True
%reload_ext autoreload
import io
import skimage
from skimage.transform import resize
from itertools import groupby
from tqdm.auto import tqdm
import gc
import seaborn as sns
sns.set()
!pip install torchsummary
from torchsummary import summary
print('Pytorch version:', torch.__version__)

Pytorch version: 1.5.1+cu92


In [ ]:
CROP_SIZE      = 44
UPSCALE_FACTOR = 4
NUM_EPOCHS     = 6

In [ ]:
from os import listdir
from os.path import join

from PIL import Image
from torch.utils.data.dataset import Dataset
from torchvision.transforms import Compose, RandomCrop, ToTensor, ToPILImage, CenterCrop, Resize

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'])

def calculate_valid_crop_size(crop_size, upscale_factor):
    return crop_size - (crop_size % upscale_factor)

def plain_transform():
    return Compose([
        ToTensor(),
    ])

def train_hr_transform(crop_size):
    return Compose([
        RandomCrop(crop_size),
        ToTensor(),
    ])

def train_lr_transform(crop_size, upscale_factor):
    return Compose([
        ToPILImage(),
        Resize(crop_size // upscale_factor, interpolation=Image.BICUBIC),
        ToTensor()
    ])

def display_transform():
    return Compose([
        ToPILImage(),
        Resize(400),
        CenterCrop(400),
        ToTensor()
    ])

# Images are read from directory.    
# 44, 4 -> parameters passed to 'calculate_crop_size' function. Both parameters are configured.
# Example : Image Input Size -> (127, 224)
# crop_size = (44 - 44%4) = 44
# hr_image.size -> (44,44) ->  i.e. crop size we calculated. We are cropping the hr_image based on crop size from input image.
# lr_image.size -> 44 // 4 = 11 -> (11,11)
# Note : Train data prep is different from validation data prep. In train data, hr_img is significantly down-sized. This is to reduce the training time.

class TrainDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, crop_size, upscale_factor):
        super(TrainDatasetFromFolder, self).__init__()
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]
        crop_size = calculate_valid_crop_size(crop_size, upscale_factor)
        self.plain_transform = plain_transform()
        self.hr_transform = train_hr_transform(crop_size)
        self.lr_transform = train_lr_transform(crop_size, upscale_factor)

    def __getitem__(self, index):
        train_image = self.plain_transform(Image.open(self.image_filenames[index]))
        hr_image = self.hr_transform(Image.open(self.image_filenames[index]))
        lr_image = self.lr_transform(hr_image)
        return lr_image, hr_image

    def __len__(self):
        return len(self.image_filenames)

class ValDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, upscale_factor):
        super(ValDatasetFromFolder, self).__init__()
        self.upscale_factor = upscale_factor
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]

# hr_image -> Original image we are giving from validation dataset. We calculate crop size & then again modify same 'hr_image' by CenterCrop from hr_image we started with.
# lr_image -> This is reduced version of original image supplied. Dimension will be crop size//upscale factor
# hr_restore_img -> This is merely resizing the lr_image to make it same size as hr_image. This will be the lr image considered for loss calculations

# Example 1
#   Orig Image size -> (224, 150)
#   150, 4 -> parameters passed to 'calculate_crop_size' function i.e. minimum of original image dimension & upscale factor that we set (in this case 4)
#   crop_size = (150 - 150%4) = 148
#   lr_scale -> 148/4 = 37 i.e. lr_image size will (37, 37)
#   hr_scale -> 148, so hr_image size will be (148,148) i.e. crop size we calculated
#   hr_restore_img_size = (148, 148) because this is mere resizing of lr_image

# Example 2 
#   Orig Image size -> (224, 224)
#   224, 4 -> parameters passed to 'calculate_crop_size' function  i.e. minimum of original image dimension & upscale factor that we set (in this case 4)
#   crop_size = (224 - 224%4) = 224
#   lr_scale -> 224/4 = 56 i.e. lr_image size will (56, 56)
#   hr_scale -> 224, so hr_image size will be (224,224) i.e. crop size we calculated
#   hr_restore_img_size = (224, 224) because this is mere resizing of lr_image

    def __getitem__(self, index):
        hr_image = Image.open(self.image_filenames[index])
        w, h = hr_image.size
        crop_size = calculate_valid_crop_size(min(w, h), self.upscale_factor)
        lr_scale = Resize(crop_size // self.upscale_factor, interpolation=Image.BICUBIC)
        hr_scale = Resize(crop_size, interpolation=Image.BICUBIC)
        hr_image = CenterCrop(crop_size)(hr_image)
        lr_image = lr_scale(hr_image)
        hr_restore_img = hr_scale(lr_image)
        return ToTensor()(lr_image), ToTensor()(hr_restore_img), ToTensor()(hr_image)

    def __len__(self):
        return len(self.image_filenames)

In [ ]:
train_set    = TrainDatasetFromFolder('/content/gdrive/My Drive/EVA4P2_S8_Data/train_data', crop_size=CROP_SIZE, upscale_factor=UPSCALE_FACTOR)
val_set      = ValDatasetFromFolder('/content/gdrive/My Drive/EVA4P2_S8_Data/valid_data', upscale_factor=UPSCALE_FACTOR)
train_loader = DataLoader(dataset=train_set, num_workers=4, batch_size=64, shuffle=True)
val_loader   = DataLoader(dataset=val_set, num_workers=4, batch_size=1, shuffle=False)

In [ ]:
import math

class Generator(nn.Module):
    def __init__(self, scale_factor):
        upsample_block_num = int(math.log(scale_factor, 2))

        super(Generator, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=9, padding=4),
            nn.PReLU()
        )
        self.block2 = ResidualBlock(64)
        self.block3 = ResidualBlock(64)
        self.block4 = ResidualBlock(64)
        self.block5 = ResidualBlock(64)
        self.block6 = ResidualBlock(64)
        self.block7 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64)
        )
        block8 = [UpsampleBLock(64, 2) for _ in range(upsample_block_num)]
        block8.append(nn.Conv2d(64, 3, kernel_size=9, padding=4))
        self.block8 = nn.Sequential(*block8)

    def forward(self, x):
        block1 = self.block1(x)
        block2 = self.block2(block1)
        block3 = self.block3(block2)
        block4 = self.block4(block3)
        block5 = self.block5(block4)
        block6 = self.block6(block5)
        block7 = self.block7(block6)
        block8 = self.block8(block1 + block7)

        return (torch.tanh(block8) + 1) / 2

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(512, 1024, kernel_size=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(1024, 1, kernel_size=1)
        )

    def forward(self, x):
        batch_size = x.size(0)
        return torch.sigmoid(self.net(x).view(batch_size))

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x):
        residual = self.conv1(x)
        residual = self.bn1(residual)
        residual = self.prelu(residual)
        residual = self.conv2(residual)
        residual = self.bn2(residual)

        return x + residual

class UpsampleBLock(nn.Module):
    def __init__(self, in_channels, up_scale):
        super(UpsampleBLock, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels * up_scale ** 2, kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(up_scale)
        self.prelu = nn.PReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.pixel_shuffle(x)
        x = self.prelu(x)
        return x

In [ ]:
netG = Generator(UPSCALE_FACTOR)
print('# generator parameters:', sum(param.numel() for param in netG.parameters()))
netD = Discriminator()
print('# discriminator parameters:', sum(param.numel() for param in netD.parameters()))

# generator parameters: 734219
# discriminator parameters: 5215425


In [ ]:
from torchvision.models.vgg import vgg16

class GeneratorLoss(nn.Module):
    def __init__(self):
        super(GeneratorLoss, self).__init__()
        vgg = vgg16(pretrained=True)
        loss_network = nn.Sequential(*list(vgg.features)[:31]).eval()
        for param in loss_network.parameters():
            param.requires_grad = False
        self.loss_network = loss_network
        self.mse_loss = nn.MSELoss()
        self.tv_loss = TVLoss()

    def forward(self, out_labels, out_images, target_images):
        # Adversarial Loss -> Calculates difference between 1 and value returned by discriminator (fake_out) after evaluating the fake img generated by generator
        adversarial_loss = torch.mean(1 - out_labels)
        # Perception Loss -> This is VGG loss between fake_img (sr) and real_img(hr)
        perception_loss = self.mse_loss(self.loss_network(out_images), self.loss_network(target_images))
        # Image Loss -> This is MSE loss between fake img and real_img
        image_loss = self.mse_loss(out_images, target_images)
        # TV Loss  -> Total Variation Loss
        tv_loss = self.tv_loss(out_images)
        return image_loss + 0.001 * adversarial_loss + 0.006 * perception_loss + 2e-8 * tv_loss

# TVLoss -> Total Variation Loss
# The total variation is the sum of the absolute differences for neighboring pixel-values in the input images. This measures how much noise is in the images.
# TV loss is getting fake_img generated(sr_img) as input

class TVLoss(nn.Module):
    def __init__(self, tv_loss_weight=1):
        super(TVLoss, self).__init__()
        self.tv_loss_weight = tv_loss_weight

    def forward(self, x):
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h = self.tensor_size(x[:, :, 1:, :])                          # If sr_img size is (64, 3, 44, 44), we are passing tensor as (64,3,43,44) i.e discarding first row. This is to facilitate vertical grad calc.
        count_w = self.tensor_size(x[:, :, :, 1:])                          # If sr_img size is (64, 3, 44, 44), we are passing tensor as (64,3,44,43) i.e discarding first column. This is to facilitate horizontal grad calc.
        h_tv = torch.pow((x[:, :, 1:, :] - x[:, :, :h_x - 1, :]), 2).sum()  # Difference of neighbouring pixel values using rows i.e. Calculating vertical gradient.(Refer EVA4-P1-S1)
        w_tv = torch.pow((x[:, :, :, 1:] - x[:, :, :, :w_x - 1]), 2).sum()  # Difference of neighbouring pixel values using columns i.e. calculation horizontal gradient.
        return self.tv_loss_weight * 2 * (h_tv / count_h + w_tv / count_w) / batch_size

    @staticmethod
    def tensor_size(t):     
        return t.size()[1] * t.size()[2] * t.size()[3]        

In [ ]:
generator_criterion = GeneratorLoss()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [ ]:
if torch.cuda.is_available():
   netG.cuda()
   netD.cuda()
   generator_criterion.cuda()

optimizerG = optim.Adam(netG.parameters())
optimizerD = optim.Adam(netD.parameters())

results = {'d_loss': [], 'g_loss': [], 'd_score': [], 'g_score': [], 'psnr': [], 'ssim': []}

In [ ]:
summary(netD, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           1,792
         LeakyReLU-2         [-1, 64, 128, 128]               0
            Conv2d-3           [-1, 64, 64, 64]          36,928
       BatchNorm2d-4           [-1, 64, 64, 64]             128
         LeakyReLU-5           [-1, 64, 64, 64]               0
            Conv2d-6          [-1, 128, 64, 64]          73,856
       BatchNorm2d-7          [-1, 128, 64, 64]             256
         LeakyReLU-8          [-1, 128, 64, 64]               0
            Conv2d-9          [-1, 128, 32, 32]         147,584
      BatchNorm2d-10          [-1, 128, 32, 32]             256
        LeakyReLU-11          [-1, 128, 32, 32]               0
           Conv2d-12          [-1, 256, 32, 32]         295,168
      BatchNorm2d-13          [-1, 256, 32, 32]             512
        LeakyReLU-14          [-1, 256,

In [ ]:
summary(netG, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]          15,616
             PReLU-2         [-1, 64, 128, 128]               1
            Conv2d-3         [-1, 64, 128, 128]          36,928
       BatchNorm2d-4         [-1, 64, 128, 128]             128
             PReLU-5         [-1, 64, 128, 128]               1
            Conv2d-6         [-1, 64, 128, 128]          36,928
       BatchNorm2d-7         [-1, 64, 128, 128]             128
     ResidualBlock-8         [-1, 64, 128, 128]               0
            Conv2d-9         [-1, 64, 128, 128]          36,928
      BatchNorm2d-10         [-1, 64, 128, 128]             128
            PReLU-11         [-1, 64, 128, 128]               1
           Conv2d-12         [-1, 64, 128, 128]          36,928
      BatchNorm2d-13         [-1, 64, 128, 128]             128
    ResidualBlock-14         [-1, 64, 1

In [ ]:
from math import exp

import torch.nn.functional as F
from torch.autograd import Variable

def gaussian(window_size, sigma):    # Creates a gaussian tensor of size 11 eg: torch.Size([11])
    gauss = torch.Tensor([exp(-(x - window_size // 2) ** 2 / float(2 * sigma ** 2)) for x in range(window_size)])
    return gauss / gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)  # Adds one more dimension to gaussian tensor - torch.Size([11, 1])
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)   # mm is matrix multiplication. Also adds 2 more dimensions - torch.Size([1, 1, 11, 11])

    # expand     -> Returns a new view of the self tensor with singleton dimensions expanded to a larger size
    # contiguous -> It is like transpose but with seperate memory
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous()) # Changes window size as torch.Size([3, 1, 11, 11])                                                                                     
    return window

# Example used to explain comments below: img1.size (sr or Fake) -> torch.Size([1, 3, 144, 144]), img2.size (hr or GT) -> torch.Size([1, 3, 144, 144])

def _ssim(img1, img2, window, window_size, channel, size_average=True):    
    mu1 = F.conv2d(img1, window, padding=window_size // 2, groups=channel)   # Conv2d parms: torch.Size([1, 3, 144, 144]), torch.Size([3, 1, 11, 11]), padding= 5, groups = 3
                                                                             # conv2d returns mu1 & mu2 of size torch.Size([1, 3, 144, 144])    
    mu2 = F.conv2d(img2, window, padding=window_size // 2, groups=channel)

    mu1_sq = mu1.pow(2)       # mu1_sq and mu2_sq sizes : torch.Size([1, 3, 144, 144])
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=window_size // 2, groups=channel) - mu1_sq  # sigma1_sq.size: torch.Size([1, 3, 144, 144])
    sigma2_sq = F.conv2d(img2 * img2, window, padding=window_size // 2, groups=channel) - mu2_sq  # sigma2_sq.size: torch.Size([1, 3, 144, 144])
    sigma12 = F.conv2d(img1 * img2, window, padding=window_size // 2, groups=channel) - mu1_mu2
    
    C1 = 0.01 ** 2
    C2 = 0.03 ** 2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))  # ssim_map.size : torch.Size([1, 3, 144, 144])

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)

class SSIM(torch.nn.Module):
    def __init__(self, window_size=11, size_average=True):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average
        self.channel = 1
        self.window = create_window(window_size, self.channel)

    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()

        if channel == self.channel and self.window.data.type() == img1.data.type():
            window = self.window
        else:
            window = create_window(self.window_size, channel)

            if img1.is_cuda:
                window = window.cuda(img1.get_device())
            window = window.type_as(img1)

            self.window = window
            self.channel = channel

        return _ssim(img1, img2, window, self.window_size, channel, self.size_average)

def ssim(img1, img2, window_size=11, size_average=True):  # This is the function that is called while training which calls -> create_window -> _ssim (img1 is sr, img2 is hr/GT)
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel)
    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)   # type_as -> Returns this tensor cast to the type of the given tensor.

    return _ssim(img1, img2, window, window_size, channel, size_average)

In [ ]:
import argparse
import os
from math import log10

import pandas as pd
import torch.optim as optim
import torch.utils.data
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm

#import pytorch_ssim
#from data_utils import TrainDatasetFromFolder, ValDatasetFromFolder, display_transform
#from loss import GeneratorLoss
#from model import Generator, Discriminator

In [ ]:
netD_prev_save = f'/content/gdrive/My Drive/EVA4P2_S8_Data/Weights/netD_5_20201011082115.pt'
netD.load_state_dict(torch.load(netD_prev_save))
netG_prev_save = f'/content/gdrive/My Drive/EVA4P2_S8_Data/Weights/netG_5_20201011082115.pt'
netG.load_state_dict(torch.load(netG_prev_save))

<All keys matched successfully>

In [ ]:
    for epoch in range(1, NUM_EPOCHS + 1):

        running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 'd_score': 0, 'g_score': 0}
        print('epoch',epoch)
    
        netG.train()
        netD.train()
        for data, target in train_loader:    # data is lr image, target is hr image. eg: lr image (64,3,11,11) & hr image (64,3,44,44)
            g_update_first = True
            batch_size = data.size(0)
            running_results['batch_sizes'] += batch_size
    
            ############################
            # (1) Update D network: maximize D(x)-1-D(G(z))
            ###########################
            real_img = Variable(target)   # We are using variable to ensure back-propagation reaches the input hr image
            if torch.cuda.is_available():
                real_img = real_img.cuda()
            z = Variable(data)           # We are using variable to ensure back-propagation reaches the input lr image 
            if torch.cuda.is_available():
                z = z.cuda()
            fake_img = netG(z)          # lr img is the input to generator which will create an sr image
    
            netD.zero_grad()
            real_out = netD(real_img).mean()
            fake_out = netD(fake_img).mean()
            d_loss = 1 - real_out + fake_out
            d_loss.backward(retain_graph=True)
            optimizerD.step()
    
            ############################
            # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss
            ###########################
            netG.zero_grad()
            ############################################################
            ## The two lines below are added to prevent runetime error! ##
            fake_img = netG(z)
            fake_out = netD(fake_img).mean()
            ############################################################
            g_loss = generator_criterion(fake_out, fake_img, real_img)    # fake_img will be same size as real_img (i.e. hr_img used for training) eg: (64,3,44,44)
            g_loss.backward()
            
            fake_img = netG(z)
            fake_out = netD(fake_img).mean()
            
            
            optimizerG.step()

            # loss for current batch before optimization 
            running_results['g_loss']  += g_loss.item() * batch_size
            running_results['d_loss']  += d_loss.item() * batch_size
            running_results['d_score'] += real_out.item() * batch_size
            running_results['g_score'] += fake_out.item() * batch_size

        if epoch % 3 == 0:
            loss_d  = running_results['d_loss'] / running_results['batch_sizes']
            loss_g  = running_results['g_loss'] / running_results['batch_sizes']
            d_score = running_results['d_score'] / running_results['batch_sizes']
            g_score = running_results['g_score'] / running_results['batch_sizes']
            print(f'Training ~ Epoch - {epoch}/{NUM_EPOCHS}, Loss_D: {loss_d:.4f}, Loss_G: {loss_g:.4f}, D(x) Score: {d_score:.4f}, D(G(z)) Score: {g_score:.4f}')            
    
        netG.eval()
        out_path     = f'/content/gdrive/My Drive/EVA4P2_S8_Data/Results/'
        path_name_wt = f'/content/gdrive/My Drive/EVA4P2_S8_Data/Weights/'
        
        with torch.no_grad():
            valing_results = {'mse': 0, 'ssims': 0, 'psnr': 0, 'ssim': 0, 'batch_sizes': 0}
            val_images = []

            # Image sizes of validation differs from training. Refer TrainDatasetFromFolder class for more details.
            # val_lr -> Low resolution image, val_hr_restore -> Resized version of Low resolution image, val_hr -> Cropped version of original image

            for val_lr, val_hr_restore, val_hr in val_loader:
                batch_size = val_lr.size(0)
                valing_results['batch_sizes'] += batch_size
                lr = val_lr
                hr = val_hr
                if torch.cuda.is_available():
                    lr = lr.cuda()
                    hr = hr.cuda()
                sr = netG(lr)
        
                batch_mse = ((sr - hr) ** 2).data.mean()
                valing_results['mse'] += batch_mse * batch_size
                #batch_ssim = pytorch_ssim.ssim(sr, hr).item()
                batch_ssim = ssim(sr, hr).item()
                valing_results['ssims'] += batch_ssim * batch_size
                valing_results['psnr'] = 10 * log10((hr.max()**2) / (valing_results['mse'] / valing_results['batch_sizes']))
                valing_results['ssim'] = valing_results['ssims'] / valing_results['batch_sizes']
        
                # This is for display, left will be having LR image (hr_restore is a resized version of LR img only)
                #                    middle will be having HR image (this is the original validation image - ground truth)
                #                     right will be having SR image (this is the generated img, our aim is to make this better than GT)
                val_images.extend(
                    [display_transform()(val_hr_restore.squeeze(0)), display_transform()(hr.data.cpu().squeeze(0)),
                     display_transform()(sr.data.cpu().squeeze(0))])
            
            val_images = torch.stack(val_images)                             # Concatenates sequence of tensors. All tensors need to be of the same size. 
                                                                             # eg: If val dataset has 20 images, then [60, 3, 400, 400] i.e. 20 + 20 + 20 = 60
            val_images = torch.chunk(val_images, val_images.size(0) // 15)   # Splits a tensor into a specific number of chunks. 
                                                                             # We are displaying 5 images, 5 rows - each row having LR, HR, SR
            
            if epoch % 5 == 0:
                psnr_val = valing_results['psnr']
                ssim_val = valing_results['ssim']
                print(f'Validation ~ Epoch - {epoch}/{NUM_EPOCHS}, PSNR: {psnr_val:.4f}, SSIM: {ssim_val:.4f}')

                for image in val_images:
                    t = datetime.now()
                    time_stamp = t.strftime("%Y")+t.strftime("%m")+t.strftime("%d")+t.strftime("%H")+t.strftime("%M")+t.strftime("%S")                
                    image = utils.make_grid(image, nrow=3, padding=5)
                    #utils.save_image(image, out_path + 'epoch_%d_index_%d.png' % (epoch, index), padding=5)
                    utils.save_image(image, f'{out_path}SRGAN_{epoch}_{time_stamp}.png', padding=5)
    
        # save model parameters
        #torch.save(netG.state_dict(), 'epochs/netG_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
        #torch.save(netD.state_dict(), 'epochs/netD_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
        # save loss\scores\psnr\ssim
        results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
        results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
        results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
        results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
        results['psnr'].append(valing_results['psnr'])
        results['ssim'].append(valing_results['ssim'])
    
        if epoch % 10 == 0:
            out_path = f'/content/gdrive/My Drive/EVA4P2_S8_Data/Statistics/'
            data_frame = pd.DataFrame(
                data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
                      'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': results['ssim']})
                #index=range(1, epoch + 1))
            data_frame.to_csv(out_path + 'srf_' + str(UPSCALE_FACTOR) + '_train_results.csv', index_label='Epoch')

        ### Keep the model in Gpu & Save the model values in intermittent epochs
        if epoch % 5 == 0 or epoch == (NUM_EPOCHS-1):
            t = datetime.now()
            time_stamp = t.strftime("%Y")+t.strftime("%m")+t.strftime("%d")+t.strftime("%H")+t.strftime("%M")+t.strftime("%S")         
            torch.save(netG.state_dict(),f'{path_name_wt}netG_{epoch}_{time_stamp}.pt')
            torch.save(netD.state_dict(),f'{path_name_wt}netD_{epoch}_{time_stamp}.pt')                              
            print(f'GPU models saved in epoch {epoch}/{NUM_EPOCHS}')

        ### Convert the model to CPU & save the model values on final epoch    
        if epoch == (NUM_EPOCHS-1):              
            t = datetime.now()
            time_stamp = t.strftime("%Y")+t.strftime("%m")+t.strftime("%d")+t.strftime("%H")+t.strftime("%M")+t.strftime("%S")
            netG.eval()
            netD.eval()
            netG.to('cpu')
            netD.to('cpu')
            traced_netD = torch.jit.trace(netD,torch.randn(1,3,128,128))      
            traced_netD.save(f'{path_name_wt}netD_CPU_{epoch}_{time_stamp}.pt')
            traced_netG = torch.jit.trace(netG,torch.randn(1,3,128,128))      
            traced_netG.save(f'{path_name_wt}netG_CPU_{epoch}_{time_stamp}.pt')             
            print(f' **** CPU models Saved in epoch:{epoch+1}/{NUM_EPOCHS}')
            netG.cuda()
            netD.cuda()     

epoch 1
epoch 2
epoch 3
Training ~ Epoch - 3/6, Loss_D: 1.0074, Loss_G: 0.0046, D(x) Score: 0.6409, D(G(z)) Score: 0.6465
epoch 4
epoch 5
Validation ~ Epoch - 5/6, PSNR: 25.4787, SSIM: 0.8367
GPU models saved in epoch 5/6
 **** CPU models Saved in epoch:6/6
epoch 6
Training ~ Epoch - 6/6, Loss_D: 1.0037, Loss_G: 0.0043, D(x) Score: 0.8359, D(G(z)) Score: 0.8416
